In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import copy
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [113]:
G = nx.random_graphs.erdos_renyi_graph(500, 0.6)

In [82]:
def grarep(G, k, d, lmbd):
    S = nx.to_numpy_matrix(G)
    N = len(G.nodes)
    dictM = {}
    beta = lmbd/N
    for i in range(1, k+1):
        if i == 1:
            D = np.diag(np.sum(S, axis = 0).A1)
            D1 = np.linalg.inv(D)
            A = D1 @ S
            curA = copy.deepcopy(A)
        else:
            curA = curA @ A
        gamma = np.repeat(np.sum(curA, axis = 0), [A.shape[0]], axis = 0)
        Y = np.log(np.multiply(curA, 1/gamma))-np.log(beta)
        X = Y
        X[X < 0] = 0
        U, D, VT = np.linalg.svd(X)
        Ud = U[:, :d]
        Dd = np.diag(D)[:d, :d]
        W = Ud @ np.sqrt(Dd)
        dictM["W"+ str(i)] = W
    return dictM
    

In [93]:
ret = grarep(G, 3, 4, 4)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app


In [115]:
def HOPE(G, d, proximity, *argv):
    A = nx.to_numpy_matrix(G)
    N = len(G.nodes)
    assert(d <= n)
    if proximity == "Katz":
        par = argv[0]
        Mg = np.identity(N) - par*A
        Ml = par*A
    elif proximity == "RPR":
        par = argv[0]
        D = np.diag(np.sum(A, axis = 0).A1)
        D1 = np.linalg.inv(D)
        P = D1 @ A
        Mg = np.identity(N) - par*P
        Ml = (1-par)*np.identity(N)
    elif proximity == "CN":
        Mg = np.identity(N)
        Ml = A @ A
    elif proximity == "AA":
        D = np.diag([1/(np.sum(A[:, i])+np.sum(A[i, :])) for i in range(N)])
        Mg = np.identity(N)
        Ml = A @ D @ A
    else:
        print("Unknown proximity function. Using CN")
        Mg = np.identity(N)
        Ml = A @ A
    #Partial GSVD/JDGSVD may be implemented here
    S = np.linalg.inv(Mg) @ Ml
    U, D, VT = np.linalg.svd(S)
    Ds = np.sqrt(np.diag(D)[:d, :d])
    Us = U[:, :d] @ Ds
    Ut = VT.T[:, :d] @ Ds
    return {"Us": Us, "Ut" : Ut}

In [117]:
uu = HOPE(G, 3, "Katz", 0.3)